In [ ]:
%%writefile inventario_setor_sujo.csv
Patrimonio,Equipamento,Data_Aquisicao,Preco,Proprietario
1245,Notebook,2023-01-25,130.24,Julio Fernandes
1245,Notebook,2023-01-25,130.24,Julio Fernandes
1377,Nobreak,2023-04-12,98.99,Julio Fernandes
1590,Monitor ,2023-02-10,120,Julio Fernandes
2983,HD Exerno,21/03/2024,49.27,Bernardo Campos
1760,monitor,2023-02-20,120.00,Bernardo Campos
1320,Notebook,2023-05-10,130.24,Bernardo Campos
2770,Nobreak,2024-07-12,NaN,Claudia Silva
1150,Notebook,2023-01-20,130.24,Claudia  Silva
1842,TECLADO,2023-03-15,35.50,Julio Fernandes
1901,Mouse,2023-03-16,22,90,Julio Fernandes
2055,Impressora,2024-01-08,R$ 210.00,Julio Fernandes
2210,Monitor,,119.99,Ana Ribeiro
2234,Laptop,2023-06-20,132.50,Ana Ribeiro
2259,Headset,2023-07-02,58.75,Ana Ribeiro
24O1,Roteador,2024-02-11,89.90,Bernardo Campos
2488,Teclado,19/08/2023,36.00,Bernardo Campos
2550,Mause,2023-08-19,24.50,Bernardo Campos
2603,Scanner,2024-03-05,175.30,Claudia Silva
2689,Monitor,2023-09-14,118.00,Claudia Silva
2701,Teclado,2023-09-14,34.90,Claudia Silva
2810,Notebook,2024-04-22,135.00,Marcos Oliveira
2844,Mouse,2024-04-22,23.99,Marcos Oliveira
2899,Webcam,2024-05-01,64.80,Marcos Oliveira
3002,Monitor,2023-10-10,121.45,Renata Lopes
3033,Headset,2023-10-11,59.90,Renata Lopes
3098,Roteador,2024-06-03,92.00,Renata Lopes
3150,Notebook,2024-06-15,138.20,Carlos Menezes
3197,HD Externo,2024-06-18,52.75,Carlos Menezes
3221,Mouse,2024-06-18,25.00,Carlos Menezes
3305,Impressora,2024-07-01,215.60,Patricia Nunes
3340,,2024-07-02,178.00,Patricia Nunes
3388,Monitor,2023-11-25,122.00,Patricia Nunes
1903,Mouse,2023-06-18,18.90,Marcos Oliveira
2011,Impressora,05-01-2024,210.75,Ana Paula
2144,Scanner,2024-02-14,189.30,Ana Paula
2259,Notebook,2024-03-02,145.99,Rafael Mendes
2380,Monitor,2024-03-02,122.00,Rafael Mendes
2467,Switch,2023-11-22,85.40,Patricia Nunes
2521,Roteador,2023-11-22,99.90,Patricia Nunes
2678,Projetor,2024-05-10,32000,Lucas Ribeiro
2712,Estabilizador,2024-05-10,65.80,Lucas Ribeiro
2899,Tablet,2023-08-30,110.45,Renata Lopes
2950,Monitor,2023-09-01,118.00,Renata Lopes
3041,Notebook,2024-06-15,150.00,Thiago Martins
3176,HD Interno,2024-06-15,72.35,Thiago Martins
3290,Webcam,2023-10-05,54.90,Fernanda Costa
3348,Fone de Ouvido,2023-10-05,39.99,Fernanda Costa
3482,Servidor,2024-07-01,980.00,Daniel Rocha
3527,Nobreak,2024-07-01,102.50,Daniel Rocha
3691,Leitor de Codigo de Barras,2023-12-12,140.00,Carla Menezes
3720,Impressora Termica,2023-12-12,260.00,Carla Menezes

In [4]:
import pandas as pd
from sqlalchemy import create_engine, Column, String, Float, Integer, Text
from sqlalchemy.orm import declarative_base, sessionmaker

In [ ]:
# transformando a planilha em um DataFrame
df = pd.read_csv('/content/inventario_setor_sujo.csv',sep=',',on_bad_lines='warn')

# limpeza de dados nulos
df = df.dropna()

# limpeza de dados duplicados
df = df.drop_duplicates(keep='first')

# limpeza e padronização dos dados
df['Data_Aquisicao'] = pd.to_datetime(df['Data_Aquisicao'],format='mixed')
df['Equipamento'] = df['Equipamento'].str.strip().str.title()
df['Preco'] = df['Preco'].str.replace(',','.').str.replace('R$','')

In [ ]:
# conexão com o banco de dados
engine = create_engine('sqlite:///inventario_setor.bd')
Base = declarative_base()
_Sessao = sessionmaker()

# classe base para exportar os dados
class Equipamento(Base):
  __tablename__ = 'equipamentos'

  patrimonio = Column(Integer, primary_key=True, unique=True, nullable=False)
  equipamento = Column(String, nullable=False)
  data_aquisicao = Column(Text, nullable=False)
  preco = Column(Float, nullable=False)
  proprietario = Column(String, nullable=False)

  def __repr__(self):
    return self.equipamento
# linkagem da classe com o banco
  Base.metadata.create_all(engine)

# conversão do DataFrame para o banco de dados
df.to_sql(name='equipamentos',con=engine,if_exists='append',index=False, method='multi')

# obtendo dados do banco
query_sql = """
SELECT Equipamento, COUNT(*) as Quantidade
FROM equipamentos
GROUP BY Equipamento
ORDER BY Quantidade DESC
"""

# executa no banco e traz o resultado para um DataFrame
df_quant = pd.read_sql(query_sql, con=engine)
print(df_quant)